<a href="https://colab.research.google.com/github/tamaramrt/Mamo.ia/blob/main/adapta%C3%A7%C3%A3o_codigo_wylk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalação das Bibliotecas:**

*Instalação das bibliotecas utilizadas no código:*

*   *[fpdf](https://pyfpdf.readthedocs.io/en/latest): Geração de PDFs com Python.*
*   *[gradio](https://www.gradio.app/docs): Interface gráfica para melhor interação.*
*   *[matplotlib](https://matplotlib.org): Visualização de dados.*
*   *[numpy](https://numpy.org/doc): Tratamento numérico de de arrays.*
*   *[opencv-python](https://docs.opencv.org/4.x): Tratamento de imagens e visão computacional.*
*   *[pandas](https://pandas.pydata.org/docs): Tratamento com estruturas de dados, DataFrames e tabelas.*
*   *[pillow](https://pillow.readthedocs.io/en/stable): Tratamento de imagens.*
*   *[pyTorch](https://docs.pytorch.org/docs/stable/index.html): Ferramentas de redes neurais.*
*   *[pywavelets](https://pywavelets.readthedocs.io/en/latest/#): Tratamento de imagens e sinais, com a transformada wavelet.*
*   *[seaborn](https://seaborn.pydata.org): Visualização de dados.*
*   *[scikit-learn](https://scikit-learn.org/stable): Ferramentas de Machine Learning, regressão linear, clustering, etc.*
*   *[tensorflow](https://www.tensorflow.org/api_docs): Ferramentas de Inteligência Artificial e Machine Learning.*

*Outras bibliotecas que já estão inclusas no Google Colab e serão utilizadas:*

*   *[google.colab](https://colab.research.google.com/notebooks/snippets/accessing_files.ipynb): Tratamento de arquivos, como upload ou Google Drive.*
*   *[io](https://docs.python.org/3/library/io.html): Tratamento de fluxo de dados de entrada e saída.*
*   *[os](https://docs.python.org/3/library/os.html): Tratamento com o sistema operacional.*
*   *[zipfile](https://docs.python.org/3/library/zipfile.html): Tratamento de arquivos do formato ZIP.*

In [ ]:
!pip install fpdf gradio matplotlib numpy opencv-python pandas pillow pywavelets scikit-learn seaborn tensorflow

# **Importação das Bibliotecas:**

In [ ]:
import cv2
import io
import os
import pywt
import zipfile

import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# import tensoflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


from fpdf import FPDF
from google.colab import files
from PIL import Image, ImageDraw, ImageFont
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torchvision import datasets

ModuleNotFoundError: No module named 'fpdf'

# **Funções Auxiliares**

In [ ]:
# Função para extrair o arquivo ZIP
def extrair_zip(caminho_zip : str, destino : str):
    """
    Parametros:
        input:
        caminho_zip [str]: Caminho do arquivo ZIP.
        destino [str]: Caminho para a pasta de destino.
    """
    with zipfile.ZipFile(file=caminho_zip, mode='r') as zip_ref:
        zip_ref.extractall(destino)

# Função para carregar e pré-processar as imagens
def carregar_imagens(caminho_pasta : str):
    """
    Parametros:
        input:
        caminho_pasta [str]: Caminho da pasta

        output:
        np.array(imagens) [np.ndarray]: Array com as imagens da pasta.
        np.array(rotulos) [np.ndarray]: Array com os rótulos das imagens.
    """

    # Lista para as imagens e rótulos
    imagens = []
    rotulos = []

    # Percorre a pasta extraída
    for pasta in os.listdir(caminho_pasta):

        # Percorre as subpastas
        caminho_pasta_imagens = os.path.join(caminho_pasta, pasta)
        if os.path.isdir(caminho_pasta_imagens):
            for imagem_nome in os.listdir(caminho_pasta_imagens):
                # Caminho da imagem
                caminho_imagem = os.path.join(caminho_pasta_imagens, imagem_nome)

                # Leitura da imagem com opencv-python
                imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)

                # Verifica se a variavel é vazia
                if imagem is None:
                    continue
                else:
                    # Redimensiona a imagem para 224x224
                    imagem = cv2.resize(imagem, (224, 224))

                    # Adiciona a imagem e rótulo a lista
                    imagens.append(imagem)
                    rotulos.append(pasta)

    # Retorno da função
    return np.array(imagens), np.array(rotulos)

# Função para aplicar a DWT
def aplicar_dwt(imagem : np.ndarray):
    """
    Parametros:
        input:
        imagem [np.ndarray]: Array do tipo NumPy representando a imagem

        output:
        np.concatenate([LL.flatten(), LH.flatten(), HL.flatten(), HH.flatten()]) [np.ndarray]: Coeficientes da DWT.
    """

    # Realizando a transformada DWT
    coeffs2 = pywt.dwt2(imagem, 'bior1.3')
    LL, (LH, HL, HH) = coeffs2
    return np.concatenate([LL.flatten(), LH.flatten(), HL.flatten(), HH.flatten()])

# **Rede Neural - LeNet:**

In [ ]:
# Definição da arquitetura da LeNet ajustada para CIFAR-100
class LeNetCIFAR(nn.Module):
    def __init__(self):
        super(LeNetCIFAR, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)  # Primeira camada convolucional para imagens RGB
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # Camada de pooling
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5) # Segunda camada convolucional
        self.fc1 = nn.Linear(50 * 5 * 5, 500)  # Primeira camada totalmente conectada
        self.fc2 = nn.Linear(500, 100)  # Camada de saída para 100 classes (cifar-100)


    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Configurações da LeNet
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.fc1 = nn.Linear(50 * 4 * 4, 500) #penultima camada (onde vai ser feita a extracao de atributos)
        self.fc2 = nn.Linear(500, 2) # nesse caso so temos duas classes (T1 e T2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 50 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# **Tratamento dos Arquivos:**

In [ ]:
# Caminhos utilizados
caminho_zip = 'mama.zip'
destino = 'mama_extraida'

# Processo de extração e preparação dos dados
extrair_zip(caminho_zip, destino)
imagens, rotulos = carregar_imagens(destino)

NameError: name 'extrair_zip' is not defined

# **Pré-Treinamento:**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformações para a normalização das imagens e conversao para tensores
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Carregar dataset CIFAR-100
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# Criar dataloaders (carregar os dados em lotes)
trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Inicializar o modelo, função de perda e otimizador
model = LeNetCIFAR().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Função para treinar o modelo
def train_model(num_epochs : int = 10):

    # Executando o modelo para cada época
    for epoch in range(num_epochs):
        running_loss = 0.0

        # Calculando as perdas
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}")

    # Salvando o modelo
    torch.save(model.state_dict(), 'lenet_cifar100.pth')
    print("Modelo salvo como 'lenet_cifar100.pth'.")

train_model(num_epochs=10)

# **Montagem do Drive:**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Extração de Atributos:**

In [ ]:
# Pré-processamento da imagem para torná-la compatível com a Lenet
def preprocessamento_imagem(image_path : str):

    transform = transforms.Compose([transforms.Resize((32, 32)),                            # Redimensiona imagem para 32x32
                                    transforms.ToTensor(),                                  # Converte a imagem em tensor
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalização
                                    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)

    return image

# Função para extrair atributos
def extrair_atributos_img(image_path : str, model):

    input_image = preprocessamento_imagem(image_path).to(device)

    # Modo avaliação do modelo
    model.eval()

    with torch.no_grad():
        # Passar pela conv1, conv2 e obter a penúltima camada
        x = model.pool(torch.relu(model.conv1(input_image)))
        x = model.pool(torch.relu(model.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)

        # Extrair os atributos da penúltima camada
        features = torch.relu(model.fc1(x))

    # Converter para lista e retornar
    return features.squeeze().cpu().numpy().tolist()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instanciar a LeNet e carregar os pesos pré-treinados
model = LeNet().to(device)
model.load_state_dict(torch.load('lenet_cifar100.pth'))  # Carrega o modelo treinado

# Caminho para a imagem de exemplo
image_path = '/content/Lesao_Benigna_T2_71.jpg'

# Extração de atributos
features = extrair_atributos_img(image_path, model)
print(f"Atributos extraídos: {features}")

NameError: name 'torch' is not defined

# **Modelo:**

# **Gráfico de Métricas:**

In [ ]:
# Avaliação
perda, acuracia = modelo.evaluate(X_test, y_test)
print(f'\n✅ Acurácia no conjunto de teste: {acuracia * 100:.2f}%\n')

# Relatório de classificação
y_pred = np.argmax(modelo.predict(X_test), axis=1)
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Matriz confusão
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title('Matriz de Confusão')
plt.tight_layout()
plt.show()

# Métricas
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, labels=np.unique(y_test))
classes = le.classes_

# Plot das métricas
plt.figure(figsize=(10, 6))
x = np.arange(len(classes))
width = 0.25


plt.bar(x - width, precision, width=width, label='Precisão', color='skyblue')
plt.bar(x, recall, width=width, label='Recall', color='orange')
plt.bar(x + width, f1, width=width, label='F1-score', color='green')

plt.xticks(x, classes, rotation=45)
plt.ylabel('Pontuação')
plt.title('Métricas por Classe')
plt.legend()
plt.tight_layout()
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.show()

# **Interface Gráfica:**